# Python egm adapter

1) Python version 2.7

So, the point of an adapter program is to communicate with the egm server (which is the cap I'm putting on top of the robot controller). To do this, you're gonna have to use google protocol buffers to serialize and deseralize the messages sent between the server and this addapter. Because RobotStudio uses v2.4.1 protocol buffers, the decision was made to also use the same version for the rest of the communication. The problem is that v2.4.1 protocol buffers aren't supported in Python 3. So... if you're going to write an adapter in python, you're gonna have to use python 2. I'm sorry for that, but it is what is it.

2) Python environment setup

Because we're cobbeling together old stuff together, there's a couple of things you might have to do to make your python2 environment work with google protocol buffers. The first to mention is that installing everything in your environment might confuse python about where the standard print output is and what the standard encoding is. I'm not completely sure why, but o fix that, I've added some resetting script at the begenning of everything. The second thing is the actual instalation of the libraries you'll need. Everything below that is commented out are the instalation commands. I suggest updating pip and setuptools first, then uninstalling google in your environment. Once you do that, try to run the force install of protocol buffers v2.4.1. That should work. If you're running these scripts in a jupyter notebook like I am, you can just uncomment the commands that start with: !{sys.executable} -m... If you're using this code to make a command line script or something, just run the command in your python environment (i.e. pip uninstall google) without the !{sys.executable} -m part. 

In [1]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

#!{sys.executable} -m pip install numpy

# you may need to uninstall google because it causes conflicts with the protobuf v2.4.1
#!{sys.executable} -m pip uninstall google

# install protobuf v2.4.1
#!{sys.executable} -m pip install protobuf==2.4.1 --force-reinstall

# you might need to upgrade your package manager stuff
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install --upgrade setuptools

import time
import threading
import numpy as np
#import scipy as sc
import socket
import test_pb2 as test_proto
import lth_egm_interface_pb2 as interface_proto
import line_sensor_pb2 as sensor

In [2]:
UDP_IP_ADDRESS = "127.0.0.1"
UDP_PORT_NO = 8081             #this is probably irrelivant 
UDP_SERVER_PORT_NO = 8080
SENSOR_PORT_NO = 12345

In [3]:
# THIS IS THE SERVER LISTENING FOR SENSOR DATA
def sensor_listener():
    sensorSock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sensorSock.bind((UDP_IP_ADDRESS, SENSOR_PORT_NO))
    runSensor = True
    print "sensor server started"
    while runSensor:
        data, addr = sensorSock.recvfrom(1024)
        if len(data) is not None:
            sensorData = sensor.LineSensor()
            sensorData.ParseFromString(data)
            print sensorData.sensedPart
        else:
            print "no sensor data" 
    return

# THIS IS THE SERVER FOR MY PROTOCOL 
def greg_protocol_client(path):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_address = (UDP_IP_ADDRESS, UDP_SERVER_PORT_NO)
    conv_criteria = 0.1 #I have no idea... just a number right now
    print "guide start"
    for target in path:
        print "target start"
        # make targer command
        target_message = make_target_command(target)
        # make sure that it won't start sending all sorts of requests nothing is returned
        start_checking_convergence = False
        # send the target command and wait for the response
        target_response = send_interface_message(server_address, target_message)
        print "target response:"
        print target_response
        # if the response is valid, start checking if the robot has reached the target
        if target_response is not None:
            start_checking_convergence = True
        while start_checking_convergence:
            position_request = make_test_request()
            position_response = send_interface_message(server_address, position_request)
            print "position response:"
            print position_response
            if position_response is not None:
                current_position = make_target(position_response.currentPosition.cartesian_x, position_response.currentPosition.cartesian_y, position_response.currentPosition.cartesian_z)
                if check_target_convergence(target, current_position, conv_criteria):
                    start_checking_convergence = False
            else:
                start_checking_convergence = False
            time.sleep(0.1) #Sleep 100 milliseconds 
    
    # WHEN DONE PRINT DONE
    print "DONE!"
    return

In [4]:
def make_test_request():
    """populate an EGM_Control message with data for a request of all current robot data"""
    request = interface_proto.EGM_Control()
    request.header.mtype = interface_proto.Header.MSGTYPE_REQUEST_ALL_VALUES
    return request

def make_target_command(target):
    """populate an EGM_Control message with data to provide a new desired target"""
    command = interface_proto.EGM_Control()
    command.header.mtype = interface_proto.Header.MSGTYPE_POS_COMMAND
    command.desiredPosition.cartesian_x = target[0]
    command.desiredPosition.cartesian_y = target[1]
    command.desiredPosition.cartesian_z = target[2]
    return command

def make_target(x, y, z):
    """make a target... simple right now, will include more than x, y, z in future... but I want to standardize what a target is"""
    # SHOULD REALLY USE NUMPY FOR ALL THIS STUFF ANYWAY... I'LL ADD THAT TO THE DO-DO LIST TOO 
    target = [x, y, z,]
    return target

def check_target_convergence(targ1, targ2, conv_criteria):
    """check the euclidean distance between the two targets and compare to convergence criteria"""
    # will check if difference between the x, y, z of target 1 and target 2 are within the distance given by the convergence criteria
    p1 = np.array([targ1[0], targ1[1], targ1[2]])
    p2 = np.array([targ2[0], targ2[1], targ2[2]])
    dist = np.linalg.norm(p1-p2)
    #print dist
    if dist <= conv_criteria:
        return True
    else:
        return False

def send_interface_message(server_address, interface_message):
    """send an EGM_Control message to the server and return the response"""
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    response = None
    try:
        sent = client_socket.sendto(interface_message.SerializeToString(), server_address)
        response_data, server = client_socket.recvfrom(4096)
        response = interface_proto.EGM_Control()
        if len(response_data) is not None:
            response.ParseFromString(response_data)
        else:
            print "no response"
    finally:
        client_socket.close()
        return response

In [5]:
# this is a test path... the plan is to use this path to control 
# the robot in position guidance
target0 = [-0.05758161, -0.056587974, 10.010103493 ]
target1 = [299.94241839, -0.056587974, 10.010103493]
target2 = [299.94241839, -200.056587974, 10.010103493]
target3 = [213.474183961, -200.056587974, 10.010103493]
target4 = [190.254237086, -126.401926961, 10.010103493]
target5 = [122.459604485, -163.389260093, 10.010103493]
target6 = [59.039406438, -163.389260093, 10.010103493]
target7 = [33.084307566, -142.780034243, 10.010103493]
target8 = [-0.05758161, -0.056587974, 10.010103493]
path = []
path.append(target0)
path.append(target1)
path.append(target2)
path.append(target3)
path.append(target4)
path.append(target5)
path.append(target6)
path.append(target7)
path.append(target8)

In [6]:
t0 = [878.795086254,-300.066190776,1412.499981377]
t1 = [886.791796692,275.537350992,1412.499981377]
path2 = []
path2.append(t0)
path2.append(t1)

In [ ]:
sensor_listener = threading.Thread(target=sensor_listener)
greg

# this is the setup for the test of the dummy server: send a request 
# for everything to the server -> print out the dummy response
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = (UDP_IP_ADDRESS, UDP_SERVER_PORT_NO)
data = make_test_request()
try:
    #send data
    print "sending..."
    print data
    sent = client_socket.sendto(data.SerializeToString(), server_address)
    #receive response
    print "waiting for response..."
    response_data, server = client_socket.recvfrom(4096)
    if len(response_data) is not None:
        response = interface_proto.EGM_Control()
        response.ParseFromString(response_data)
        print response
    else:
        print "no response"
        
finally:
    print "closing client socket..."
    client_socket.close()
        
            